## Data Cleaning

Let's load the some data for TSLA. Unfortunately, this data is not quite as *clean* as our NVDA data, so we'll need to do some data wrangling. The file we're looking to load is `TSLA_2015_2024.csv`.

In [4]:
import pandas as pd

In [23]:
df = pd.read_csv("data/TSLA_2015_2024.csv")
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
df.set_index("Date", inplace=True)
df

,Close,High,Low,Open,Volume
Date,,,,,
2024-07-11,241.029999,271.000000,239.649994,263.299988,221707300.0
2020-08-13,108.066666,110.078667,104.484001,107.400002,306379500.0
2020-08-13,108.066666,110.078667,104.484001,107.400002,306379500.0
2019-10-30,21.000668,21.252666,20.664667,20.866667,144627000.0
2015-08-27,16.199333,NaN,15.387333,15.400000,114840000.0
...,...,...,...,...,...
2018-08-10,23.699333,24.000000,23.066668,23.600000,173280000.0
2021-06-21,206.943329,210.463333,202.960007,208.160004,74438100.0
2016-06-20,14.646667,14.916667,14.548667,14.633333,53332500.0


Can you see what we mean by messy? How many issues can you spot?

- Dates out of order
- Duplicate rows
- Missing values

## Ordering and Duplicates

First let's start with sorting the index.

In [21]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df
df.duplicated().sum()

np.int64(0)

Now let's focus on duplicates:

#### Tip: Method Chaining

**Method chaining** is a popular feature of pandas. It allows us to *chain* together several operations in a single line of code. For example, we can set the index, sort the data frame and drop any duplicates all at once. Notice we don't use `inplace` but rather re-assign to the original `df` variable.

```python
df = df.set_index("Date").sort_index().drop_duplicates()
```

## Not a Number (NaN)

### Exercise: Some Null Chain

Let's look at the missing or `NaN` values next. Previously, we saw that `info()` gave us some insight into how many missing values we had, but we can also use `isnull()`.

Can you chain `isnull()` with `sum()` to get a single value stating the total number of missing values in the data frame?

In [30]:
## YOUR CODE GOES HERE
condition = df.isnull().any(axis=1)
df[condition]

,Close,High,Low,Open,Volume
Date,,,,,
2015-08-27,16.199333,NaN,15.387333,15.400000,114840000.0
2018-08-02,23.302668,23.332666,21.544001,21.896000,NaN
2023-04-24,162.550003,165.649994,NaN,164.649994,140006600.0
2022-08-11,286.630005,298.236664,NaN,296.513336,70155000.0
2020-02-20,59.960667,60.799999,NaN,60.796665,264523500.0
...,...,...,...,...,...
2024-11-07,296.910004,299.750000,285.519989,NaN,117309200.0
2022-02-18,NaN,295.623322,279.203339,295.333344,68501700.0
2020-05-01,NaN,51.518002,45.535999,50.333332,487977000.0


We can find out which rows have missing data using `isnull()`, `any()` along rows and some smart *masking*.

In [35]:
df.dropna(subset="Close")
df.Volume.fillna(df.Volume.mean())
df.Close.interpolate(method="linear")
df.Close.ffill()

Date
2024-07-11    241.029999
2020-08-13    108.066666
2020-08-13    108.066666
2019-10-30     21.000668
2015-08-27     16.199333
                 ...    
2018-08-10     23.699333
2021-06-21    206.943329
2016-06-20     14.646667
2019-02-21     19.415333
2019-05-14     15.487333
Name: Close, Length: 2566, dtype: float64

Now that we've identified our missing values, the big question is how to handle them. There are many approaches to this that will vary depending on the data and the further analysis you plan to carry out.

### Exercise: Cleaning up

Notice how above we didn't actually update the `df` variable, so our DataFrame is still full of missing values. Fix all missing values applying the following rules:
- Fill missing Close by linear interpolation
- Fill missing Volume with the value from the day before
- Fill missing Open with the median Open
- Fill missing High with the Close or Open, whichever is higher
- Fill missing Low with a value 3% lower than the High


Your DataFrame `df` should have no missing values when done. Use `info()` to confirm.

**NOTE:** When changing values in a data frame, it is recommended to avoid using `inplace`, and instead re-assign the variable.

In [49]:
## YOUR CODE GOES HERE

df.Close=df.Close.interpolate(method="linear")
df.Volume=df.Volume.ffill()
df.Open=df.Open.fillna(df.Open.mean())
df.High = df.High.fillna(df[['Open', 'Close']].max(axis=1))
df.Low = df.Low.fillna(df.High*0.97)

df.isnull().sum().sum()


np.int64(0)

#### Advanced: Data Types

You may have noticed that the **Volume** column in the 2021 data frame is a `float64` instead of the `int64` *dtype* we had in the 2020 data frame. Missing values (NaN) are represented as a special case of floating point number, so all the values in **Volume** were automatically *upcast* to floats.

Ideally our columns should be of the *dtype* that most accurately represents them. This will improve performance when working with large data frames. Now that we've resolved our missing numbers, we can *cast* our trading volumes as integers.

In [50]:
df.Volume = df.Volume.astype("int64")

## Saving Data

Now that we've cleaned our data, let's save it, by writing it to a new .CSV file. We can use pandas' `to_csv()`.

In [53]:
df.to_csv("TSLA_10_clean.csv")